# Project members

**Sali Raffaele**:
- raffaele.sali@studio.unibo.it
- 000

**Zanotti Niccolò**:
- niccolo.zanotti@studio.unibo.it
- 000

**Zocco Ramazzo Marco**:
- marco.zoccoramazzo@studio.unibo.it
- 000


# Assignment Module 2: Pet Classification

The goal of this assignment is to implement a neural network that classifies images of 37 breeds of cats and dogs from the [Oxford-IIIT-Pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/). The assignment is divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.

## Dataset

The following cells contain the code to download and access the dataset you will be using in this assignment. Note that, although this dataset features each and every image from [Oxford-IIIT-Pet](https://www.robots.ox.ac.uk/~vgg/data/pets/), it uses a different train-val-test split than the original authors.

In [ ]:
#!git clone https://github.com/CVLAB-Unibo/ipcv-assignment-2.git

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torchvision.transforms import v2
from pathlib import Path
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple

In [ ]:
class OxfordPetDataset(Dataset):
    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("ipcv-assignment-2") / "dataset"
        self.split = split
        self.names, self.labels = self._get_names_and_labels()
        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img_path = self.root / "images" / f"{self.names[idx]}.jpg"
        img = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

    def _get_names_and_labels(self) -> Tuple[List[str], List[int]]:
        names = []
        labels = []

        with open(self.root / "annotations" / f"{self.split}.txt") as f:
            for line in f:
                name, label = line.replace("\n", "").split(" ")
                names.append(name),
                labels.append(int(label) - 1)

        return names, labels

In [ ]:
train_dataset1 = OxfordPetDataset(split="train")
print(len(train_dataset1))
img, label = train_dataset1[0]
print(img.size, label)

In [ ]:
def breed_from_name(name):
    return "_".join(name.split("_")[:-1])

In [ ]:
# Images if we don't apply transformations
plt.figure(figsize=(12, 6))
for i in range(6):
    img, _ = train_dataset1[i]
    name = train_dataset1.names[i]

    plt.subplot(2, 3, i + 1)
    plt.imshow(img)
    plt.title(breed_from_name(name))
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Images if we apply transformations
import torchvision.transforms as T

train_transform = T.Compose([
    T.Resize((256, 256)),
    T.RandomCrop(224),
    T.RandomHorizontalFlip(p=0.5),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = OxfordPetDataset(split="train", transform=train_transform)
test_dataset = OxfordPetDataset(split="test", transform=val_transform)
validation_dataset = OxfordPetDataset(split="val", transform=val_transform)
print("Number of samples - train:", len(train_dataset))
print("Number of classes - train:", train_dataset.get_num_classes())
print("Number of samples - test:", len(test_dataset))
print("Number of classes - test:", test_dataset.get_num_classes())
print("Number of samples - validation:", len(validation_dataset))
print("Number of classes - validation:", validation_dataset.get_num_classes())

In [ ]:
import logging
logging.getLogger('matplotlib').setLevel(logging.ERROR)

def show_samples(dataset, n=6):
    plt.figure(figsize=(12, 6))
    for i in range(n):
        img, _ = dataset[i]
        names = dataset.names[i]
        # label = dataset.labels[i]
        img = img.permute(1, 2, 0)

        plt.subplot(2, n//2, i + 1)
        plt.imshow(img)
        plt.title(f"{breed_from_name(names)}")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

show_samples(train_dataset)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)
print(images.min(), images.max())

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it from scratch on `OxfordPetDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the test split of ~60%. You are free to achieve this however you want, except for a few rules you must follow:

- Compile this notebook by displaying the results obtained by the best model you found throughout your experimentation; then show how, by removing some of its components, its performance drops. In other words, do an *ablation study* to prove that your design choices have a positive impact on the final result.

- Do not instantiate an off-the-self PyTorch network. Instead, construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you cannot use e.g. `torchvision.models.alexnet`.

- Show your results and ablations with plots, tables, images, etc. — the clearer, the better.

Don't be too concerned with your model performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded more points than a poorly experimentally validated model with higher accuracy.

## **NOTE:**
- Several strategies and network architectures were explored, with the main objective of exploiting the components and architectures presented during the course. More advanced networks, such as EfficientNet, were also developed, achieving results very close to those of a simpler ResNet-inspired network. For this reason, the network that better leverages the course topics was ultimately chosen.

## **Common Pipeline**

**Includes:**
- **Accuracy function:** takes into account different output structure when MixUp is employed or not
- **Evaluation function**
- **Residual block class:** defines the structure of the residual blocks, taking into account the presence or not of the Batch Normalization layers
- **Convolutional block class:** defines the structure of the convolutional blocks, which are employed when trying to evidence the impact of Residual blocks
- **Network class:** defines the structure of the CNN, taking into account the use of Stem layers, Batch Normalization layers, Residual layers, Pooling layers and Dropout, to measure the impact of these components
- **TrainConfig class:** defines some core parameters for the training process (like number of epochs, starting learning rate, MixUp parameter, label smoothing factor) and takes into account the use of Label Smoothing, Learning Rate Scheduler and MixUp data augmentation technique
- **build_training_components function:** defines the structure of criterion, optimizer, scheduler and mixup according to boolean and values set in the TrainConfig class
- **Train process function:** defines the pipeline for training the model, minimizing the loss function and updating weights, storing the model with the highest accuracy obtained in validation set

In [ ]:
def accuracy(outputs, labels):
    preds = outputs.argmax(dim=1)

    # If labels are one-hot / soft (MixUp case)
    if labels.ndim == 2:
        labels = labels.argmax(dim=1)

    return (preds == labels).sum()

In [ ]:
def compute_precision_recall_f1(preds, labels, num_classes):
    eps = 1e-8

    precision_list = []
    recall_list = []
    f1_list = []

    for cls in range(num_classes):
        tp = ((preds == cls) & (labels == cls)).sum().float()
        fp = ((preds == cls) & (labels != cls)).sum().float()
        fn = ((preds != cls) & (labels == cls)).sum().float()

        precision = tp / (tp + fp + eps)
        recall = tp / (tp + fn + eps)
        f1 = 2 * precision * recall / (precision + recall + eps)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    return (
        torch.mean(torch.stack(precision_list)).item(),
        torch.mean(torch.stack(recall_list)).item(),
        torch.mean(torch.stack(f1_list)).item(),
    )

In [ ]:
def evaluate(model, loader, criterion, device):
    model.eval()

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            preds = outputs.argmax(dim=1)

            # Handle one-hot / soft labels
            if labels.ndim == 2:
                labels_hard = labels.argmax(dim=1)
            else:
                labels_hard = labels

            total_loss += loss.item() * labels.size(0)
            total_correct += accuracy(outputs, labels)
            total_samples += labels.size(0)

            all_preds.append(preds)
            all_labels.append(labels_hard)

    avg_loss = total_loss / total_samples
    avg_acc = total_correct.float() / total_samples

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    precision, recall, f1 = compute_precision_recall_f1(
        all_preds,
        all_labels,
        num_classes=outputs.size(1)
    )

    return avg_loss, avg_acc.item(), precision, recall, f1

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, use_batchnorm=True):
        super().__init__()

        norm = nn.BatchNorm2d if use_batchnorm else nn.Identity

        self.conv1 = nn.Conv2d(
            in_channels, out_channels,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = norm(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(
            out_channels, out_channels,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = norm(out_channels)

        # Skip connection adjustment if shape changes
        self.skip = None
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels,
                    kernel_size=1, stride=stride, bias=False
                ),
                norm(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.skip is not None:
            identity = self.skip(identity)

        out += identity
        return self.relu(out)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, use_batchnorm=True):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_channels) if use_batchnorm else nn.Identity(),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.block(x)

In [ ]:
class Net(nn.Module):
    def __init__(
        self,
        n_classes,
        use_stem=True,
        use_residuals=True,
        use_batchnorm=True,
        use_pooling=True,
        use_dropout=True
        ):
        super().__init__()

        self.use_residuals = use_residuals

        # Stem layers
        if use_stem:
          self.stem = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
          )
        else:
          # Minimal stem: only channel lifting, no downsampling or pooling
          self.stem = nn.Sequential(
              nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
              nn.BatchNorm2d(64) if use_batchnorm else nn.Identity(),
              nn.ReLU()
          )

        # Residual blocks
        self.stage1 = self._make_stage(64, 64, num_blocks=2, stride=1, use_batchnorm=use_batchnorm)
        self.stage2 = self._make_stage(64, 128, num_blocks=2, stride=2, use_batchnorm=use_batchnorm)
        self.stage3 = self._make_stage(128, 256, num_blocks=2, stride=2, use_batchnorm=use_batchnorm)
        self.stage4 = self._make_stage(256, 512, num_blocks=2, stride=2, use_batchnorm=use_batchnorm)

        # Classifier and Pooling
        self.pool = nn.AdaptiveAvgPool2d((1, 1)) if use_pooling else None
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.2) if use_dropout else nn.Identity(),
            nn.Linear(256, n_classes)
        )

    def _make_stage(self, in_channels, out_channels, num_blocks, stride, use_batchnorm):
        if not self.use_residuals:
            # Plain conv blocks instead of residual blocks
            layers = [ConvBlock(in_channels, out_channels, stride, use_batchnorm)]
            for _ in range(1, num_blocks):
                layers.append(ConvBlock(out_channels, out_channels, 1, use_batchnorm))
            return nn.Sequential(*layers)
        else:
          layers = [ResidualBlock(in_channels, out_channels, stride, use_batchnorm)]
          for _ in range(1, num_blocks):
              layers.append(ResidualBlock(out_channels, out_channels, use_batchnorm=use_batchnorm))
          return nn.Sequential(*layers)

    def forward(self, x):
        x = self.stem(x)

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        if self.pool is not None:
          x = self.pool(x)
        else:
          x = x.mean(dim=(2, 3), keepdim=True)

        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainConfig:
    num_epochs: int = 100
    lr: float = 1e-3
    weight_decay: float = 1e-2

    use_scheduler: bool = False
    use_label_smoothing: bool = False
    label_smoothing: float = 0.1

    use_mixup: bool = False
    mixup_alpha: float = 0.2

    save_path : str = None

In [ ]:
class NoMixUp:
  def __call__(self, x, y):
    return x, y

In [ ]:
def build_training_components(model, train_loader, train_dataset, config: TrainConfig):
  criterion = nn.CrossEntropyLoss(
        label_smoothing=config.label_smoothing
        if config.use_label_smoothing
        else 0.0
    )

  optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config.lr,
        weight_decay=config.weight_decay,
    )

  scheduler = None
  if config.use_scheduler:
      scheduler = torch.optim.lr_scheduler.OneCycleLR(
          optimizer,
          max_lr=config.lr,
          epochs=config.num_epochs,
          steps_per_epoch=len(train_loader),
          pct_start=0.1,
          anneal_strategy="cos",
      )

  if config.use_mixup:
      mixup = v2.MixUp(
          alpha=config.mixup_alpha,
          num_classes=train_dataset.get_num_classes(),
      )
  else:
      mixup = NoMixUp()

  return criterion, optimizer, scheduler, mixup


In [ ]:
def train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs,
    scheduler=None,
    mixup=None,
    grad_clip=1.0,
    save_path=None,
):
  best_val_acc = 0.0
  history = {
    "train_loss": [],
    "val_loss": [],
    "train_acc": [],
    "val_acc": [],
    "val_precision": [],
    "val_recall": [],
    "val_f1": [],
    "lr": [],
  }

  for epoch in range(num_epochs):
      model.train()

      total_loss = 0.0
      total_correct = 0
      total_samples = 0

      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)

          # Apply MixUp augmentation
          images, labels = mixup(images, labels)

          optimizer.zero_grad()

          outputs = model(images)
          loss = criterion(outputs, labels)

          loss.backward()

          # Gradient clipping to prevent exploding gradients
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

          optimizer.step()
          if scheduler is not None:
            scheduler.step()  # OneCycleLR steps per batch

          total_loss += loss.item() * images.size(0)

          total_correct += accuracy(outputs, labels)
          total_samples += images.size(0)

      train_loss = total_loss / total_samples
      train_acc = total_correct.float() / total_samples

      val_loss, val_acc, val_precision, val_recall, val_f1 = evaluate(model, val_loader, criterion, device)

      # Track metrics
      history["train_loss"].append(train_loss)
      history["val_loss"].append(val_loss)
      history["train_acc"].append(train_acc.item())
      history["val_acc"].append(val_acc)
      history["val_precision"].append(val_precision)
      history["val_recall"].append(val_recall)
      history["val_f1"].append(val_f1)
      history["lr"].append(optimizer.param_groups[0]['lr'])

      if save_path is not None and val_acc > best_val_acc:
          best_val_acc = val_acc
          torch.save(model.state_dict(), save_path)

      print(
          f"Epoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.3f} | "
          f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.3f} | "
          f"LR: {optimizer.param_groups[0]['lr']:.2e}"
      )

  print(f"\nBest Validation Accuracy: {best_val_acc:.3f}")
  return best_val_acc, history

In [ ]:
def plot_accuracy(history):
    plt.figure()
    plt.plot(history["train_acc"])
    plt.plot(history["val_acc"])
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Training vs Validation Accuracy")
    plt.legend(["Train", "Validation"])
    plt.grid(True)
    plt.show()

def plot_loss(history):
    plt.figure()
    plt.plot(history["train_loss"])
    plt.plot(history["val_loss"])
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend(["Train", "Validation"])
    plt.grid(True)
    plt.show()

def plot_learning_rate(history):
    plt.figure()
    plt.plot(history["lr"])
    plt.xlabel("Iteration")
    plt.ylabel("Learning Rate")
    plt.title("Learning Rate Schedule")
    plt.grid(True)
    plt.show()

In [ ]:
CSV_PATH = "ablation_test_results.csv"
JSON_PATH = "ablation_histories.json"

In [ ]:
# Load previous CSV results if available
if os.path.exists(CSV_PATH):
    df_results = pd.read_csv(CSV_PATH)
    results = df_results.to_dict(orient="records")
else:
    results = []

# Load previous histories if available
if os.path.exists(JSON_PATH):
    with open(JSON_PATH, "r") as f:
        all_histories = json.load(f)
else:
    all_histories = {}


## **Model variant: Full model (baseline)**

**Architecture**
- Convolutional neural network implementing architectures and strategies faced during the course.
- A convolutional stem composed of a three 3×3 convolution (1st with stride=2 and others with stride=1), followed by Batch Normalization, ReLU activation, and max pooling.
- Four sequential stages operating at increasing feature dimensions (64 → 128 → 256 → 512).
- Each stage consists of two residual blocks with identity skip connections.
- When spatial resolution or channel dimensions change, skip connections are adapted using a 1×1 convolution followed by Batch Normalization.
- Global feature aggregation is performed using adaptive average pooling.
- The classifier head consists of two fully connected layers with ReLU activation and dropout.

**Normalization and regularization**
- Batch Normalization is applied after every convolution, including within residual branches and skip connections.
- Dropout is applied in the classifier to reduce overfitting.

**Training setup**
- Optimized using AdamW with weight decay.
- Learning rate scheduling is enabled via OneCycleLR with cosine annealing.
- Cross-entropy loss with label smoothing is used.
- MixUp data augmentation is applied during training.
- Gradient norm clipping is used to improve training stability.

**Purpose**
- This configuration serves as the baseline model against which all ablation studies are compared.


**NOTE:** All ablation variants modify a single component at a time while keeping the remaining architecture and training configuration identical to the baseline model.


In [ ]:
from graphviz import Source

dot_code = r"""
digraph ImprovedNet {
    rankdir=TB;
    fontname="Helvetica";
    node [shape=record, fontname="Helvetica"];

    Input [
        label="Input\n3×224×224"
    ];

    Stem [
        label="{Stem|
        Conv3×3, s=2, p=1|
        BN + ReLU|
        Conv3×3, s=1, p=1|
        BN + ReLU|
        Conv3×3, s=1, p=1|
        BN + ReLU|
        Output: 64×112×112}"
    ];

    Stage1 [
        label="{Stage 1:
        2×ResidualBlock|
        RB1:
        Conv3×3 → BN → ReLU +
        Conv3×3 → BN
        + Identity|
        RB2:
        Conv3×3 → BN → ReLU +
        Conv3×3 → BN
        + Identity|
        Output: 64x56x56}"
    ];

    Stage2 [
        label="{Stage 2:
        2×ResidualBlock|
        RB1:
        Conv3×3, s=2 + BN + ReLU +
        Conv3×3 + BN
        + Skip Conv1×1, s=2 + BN|
        Output: 128x28x28|
        RB2:
        Conv3×3 + BN + ReLU +
        Conv3×3 + BN|
        Output: 128x28x28}"
    ];

    Stage3 [
        label="{Stage 3:
        2×ResidualBlock|
        RB1:
        Conv3×3, s=2 + BN + ReLU +
        Conv3×3 + BN
        + Skip Conv1×1, s=2 + BN|
        Output: 256x14x14|
        RB2:
        Conv3×3 + BN + ReLU +
        Conv3×3 + BN|
        Output: 256x14x14}"
    ];

    Stage4 [
        label="{Stage 4:
        2×ResidualBlock|
        RB1:
        Conv3×3, s=2 + BN + ReLU +
        Conv3×3 + BN
        + Skip Conv1×1, s=2 + BN|
        Output: 512x7x7|
        RB2:
        Conv3×3 + BN + ReLU +
        Conv3×3 + BN|
        Output: 512x7x7}"
    ];

    Pool [
        label="AdaptiveAvgPool\n512×1×1"
    ];

    FC [
        label="{Classifier|
        Linear 512→256 + ReLU|
        Dropout p=0.2|
        Linear 256→N classes}"
    ];

    Output [
        label="Output\nN classes"
    ];

    Input -> Stem -> Stage1 -> Stage2 -> Stage3 -> Stage4 -> Pool -> FC -> Output;
}
"""

Source(dot_code)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_1 = Net(n_classes=train_dataset.get_num_classes()).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_1.parameters())
trainable_params = sum(p.numel() for p in Model_1.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_full_net.pth",
)

criterion, optimizer, scheduler, mixup = build_training_components(
    model = Model_1,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config,
)

best_acc, history = train_model(
    model=Model_1,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    mixup=mixup,
    device=device,
    num_epochs=config.num_epochs,
    save_path=config.save_path,
)

In [ ]:
best_model1 = Model_1

best_model1.load_state_dict(
    torch.load(config.save_path, map_location=device)
)

best_model1.to(device)

test_loss, test_acc, test_prec, test_recall, test_f1 = evaluate(best_model1, test_loader, criterion, device)

print(f"Test Loss: {test_loss:.3f}")
print(f"Test Accuracy: {test_acc:.3f}")

In [ ]:
model_name = "Baseline"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name]

# Append new results
results.append({
    "Model": model_name,
    "Test Loss": test_loss,
    "Test Accuracy": test_acc,
    "Test Precision": test_prec,
    "Test Recall": test_recall,
    "Test F1": test_f1
})

In [ ]:
# Update history
all_histories[model_name] = history

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history)
plot_loss(history)
plot_learning_rate(history)

## **Model variant: No Batch Normalization**

**Change**
- Removed all Batch Normalization layers from the network, including those in residual skip connections.

**Purpose**
- To assess the contribution of Batch Normalization to training stability and final performance.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_2 = Net(n_classes=train_dataset.get_num_classes(), use_batchnorm=False).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_2.parameters())
trainable_params = sum(p.numel() for p in Model_2.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config2 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noBN_net.pth",
)

criterion2, optimizer2, scheduler2, mixup2 = build_training_components(
    model = Model_2,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config2,
)

best_acc2, history2 = train_model(
    model=Model_2,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion2,
    optimizer=optimizer2,
    scheduler=scheduler2,
    mixup=mixup2,
    device=device,
    num_epochs=config2.num_epochs,
    save_path=config2.save_path,
)

In [ ]:
best_model2 = Model_2

best_model2.load_state_dict(
    torch.load(config2.save_path, map_location=device)
)

best_model2.to(device)

test_loss2, test_acc2, test_prec2, test_recall2, test_f12 = evaluate(best_model2, test_loader, criterion2, device)

print(f"Test Loss: {test_loss2:.3f}")
print(f"Test Accuracy: {test_acc2:.3f}")

In [ ]:
model_name2 = "No BatchNorm"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name2]

results.append({
    "Model": model_name2,
    "Test Loss": test_loss2,
    "Test Accuracy": test_acc2,
    "Test Precision": test_prec2,
    "Test Recall": test_recall2,
    "Test F1": test_f12
})

In [ ]:
all_histories[model_name2] = history2

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history2)
plot_loss(history2)
plot_learning_rate(history2)

## **Model variant: No Pooling**

**Change**
- Removed all pooling operations from the network, including max pooling in the stem and global average pooling before the classifier.

**Purpose**
- To evaluate the role of spatial downsampling and global feature aggregation in representation learning and classification performance.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_3 = Net(n_classes=train_dataset.get_num_classes(), use_pooling=False).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_3.parameters())
trainable_params = sum(p.numel() for p in Model_3.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config3 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noPOOL_net.pth",
)

criterion3, optimizer3, scheduler3, mixup3 = build_training_components(
    model = Model_3,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config3,
)

best_acc3, history3 = train_model(
    model=Model_3,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion3,
    optimizer=optimizer3,
    scheduler=scheduler3,
    mixup=mixup3,
    device=device,
    num_epochs=config3.num_epochs,
    save_path=config3.save_path,
)

In [ ]:
best_model3 = Model_3

best_model3.load_state_dict(
    torch.load(config3.save_path, map_location=device)
)

best_model3.to(device)

test_loss3, test_acc3, test_prec3, test_recall3, test_f13 = evaluate(best_model3, test_loader, criterion3, device)

print(f"Test Loss: {test_loss3:.3f}")
print(f"Test Accuracy: {test_acc3:.3f}")

In [ ]:
model_name3 = "No Pooling"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name3]


results.append({
    "Model": model_name3,
    "Test Loss": test_loss3,
    "Test Accuracy": test_acc3,
    "Test Precision": test_prec3,
    "Test Recall": test_recall3,
    "Test F1": test_f13
})

In [ ]:
all_histories[model_name3] = history3

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history3)
plot_loss(history3)
plot_learning_rate(history3)

## **Model variant: No Dropout**

**Change**
- Removed dropout from the classifier head while keeping the rest of the architecture unchanged.

**Purpose**
- To assess the impact of dropout-based regularization on overfitting and generalization performance.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_4 = Net(n_classes=train_dataset.get_num_classes(), use_dropout=False).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_4.parameters())
trainable_params = sum(p.numel() for p in Model_4.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config4 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noDOUT_net.pth",
)

criterion4, optimizer4, scheduler4, mixup4 = build_training_components(
    model = Model_4,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config4,
)

best_acc4, history4 = train_model(
    model=Model_4,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion4,
    optimizer=optimizer4,
    scheduler=scheduler4,
    mixup=mixup4,
    device=device,
    num_epochs=config4.num_epochs,
    save_path=config4.save_path,
)

In [ ]:
best_model4 = Model_4

best_model4.load_state_dict(
    torch.load(config4.save_path, map_location=device)
)

best_model4.to(device)

test_loss4, test_acc4, test_prec4, test_recall4, test_f14 = evaluate(best_model4, test_loader, criterion4, device)

print(f"Test Loss: {test_loss4:.3f}")
print(f"Test Accuracy: {test_acc4:.3f}")

In [ ]:
model_name4 = "No Dropout"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name4]


results.append({
    "Model": model_name4,
    "Test Loss": test_loss4,
    "Test Accuracy": test_acc4,
    "Test Precision": test_prec4,
    "Test Recall": test_recall4,
    "Test F1": test_f14
})

In [ ]:
all_histories[model_name4] = history4

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history4)
plot_loss(history4)
plot_learning_rate(history4)

## **Model variant: No MixUp**

**Change**
- Disabled MixUp data augmentation during training, using only standard input–label pairs.

**Purpose**
- To measure the effect of MixUp on model robustness and generalization compared to standard supervised training.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_5 = Net(n_classes=train_dataset.get_num_classes()).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_5.parameters())
trainable_params = sum(p.numel() for p in Model_5.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config5 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=False,
    save_path="best_noMIXUP_net.pth",
)

criterion5, optimizer5, scheduler5, mixup5 = build_training_components(
    model = Model_5,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config5,
)

best_acc5, history5 = train_model(
    model=Model_5,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion5,
    optimizer=optimizer5,
    scheduler=scheduler5,
    mixup=mixup5,
    device=device,
    num_epochs=config5.num_epochs,
    save_path=config5.save_path,
)

In [ ]:
best_model5 = Model_5

best_model5.load_state_dict(
    torch.load(config5.save_path, map_location=device)
)

best_model5.to(device)

test_loss5, test_acc5, test_prec5, test_recall5, test_f15 = evaluate(best_model5, test_loader, criterion5, device)

print(f"Test Loss: {test_loss5:.3f}")
print(f"Test Accuracy: {test_acc5:.3f}")

In [ ]:
model_name5 = "No MixUp"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name5]


results.append({
    "Model": model_name5,
    "Test Loss": test_loss5,
    "Test Accuracy": test_acc5,
    "Test Precision": test_prec5,
    "Test Recall": test_recall5,
    "Test F1": test_f15
})

In [ ]:
all_histories[model_name5] = history5

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history5)
plot_loss(history5)
plot_learning_rate(history5)

## **Model variant: No Data Augmentation**

**Change**
- Disabled all data augmentation techniques during training, including MixUp and any other stochastic input transformations.

**Purpose**
- To isolate the contribution of data augmentation to model generalization and performance on unseen data.

In [ ]:
train_transform2 = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset2 = OxfordPetDataset(split="train", transform=train_transform2)

train_loader2 = DataLoader(
    train_dataset2,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_6 = Net(n_classes=train_dataset.get_num_classes()).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_6.parameters())
trainable_params = sum(p.numel() for p in Model_6.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config6 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=False,
    save_path="best_noAUGM_net.pth",
)

criterion6, optimizer6, scheduler6, mixup6 = build_training_components(
    model = Model_6,
    train_loader=train_loader2,
    train_dataset=train_dataset2,
    config=config6,
)

best_acc6, history6 = train_model(
    model=Model_6,
    train_loader=train_loader2,
    val_loader=val_loader,
    criterion=criterion6,
    optimizer=optimizer6,
    scheduler=scheduler6,
    mixup=mixup6,
    device=device,
    num_epochs=config6.num_epochs,
    save_path=config6.save_path,
)

In [ ]:
best_model6 = Model_6

best_model6.load_state_dict(
    torch.load(config6.save_path, map_location=device)
)

best_model6.to(device)

test_loss6, test_acc6, test_prec6, test_recall6, test_f16 = evaluate(best_model6, test_loader, criterion6, device)

print(f"Test Loss: {test_loss6:.3f}")
print(f"Test Accuracy: {test_acc6:.3f}")

In [ ]:
model_name6 = "No Augmentation"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name6]

results.append({
    "Model": model_name6,
    "Test Loss": test_loss6,
    "Test Accuracy": test_acc6,
    "Test Precision": test_prec6,
    "Test Recall": test_recall6,
    "Test F1": test_f16
})

In [ ]:
all_histories[model_name6] = history6

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history6)
plot_loss(history6)
plot_learning_rate(history6)

## **Model variant: No Stem Layer**

**Change**
- Removed the convolutional stem (7×7 convolution, Batch Normalization, ReLU, and max pooling), feeding inputs directly into the first stage of the network.

**Purpose**
- To investigate the importance of early-stage feature extraction and aggressive spatial downsampling.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_7 = Net(n_classes=train_dataset.get_num_classes(), use_stem=False).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_7.parameters())
trainable_params = sum(p.numel() for p in Model_7.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config7 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noSTEM_net.pth",
)

criterion7, optimizer7, scheduler7, mixup7 = build_training_components(
    model = Model_7,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config7,
)

best_acc7, history7 = train_model(
    model=Model_7,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion7,
    optimizer=optimizer7,
    scheduler=scheduler7,
    mixup=mixup7,
    device=device,
    num_epochs=config7.num_epochs,
    save_path=config7.save_path,
)

In [ ]:
best_model7 = Model_7

best_model7.load_state_dict(
    torch.load(config7.save_path, map_location=device)
)

best_model7.to(device)

test_loss7, test_acc7, test_prec7, test_recall7, test_f17 = evaluate(best_model7, test_loader, criterion7, device)

print(f"Test Loss: {test_loss7:.3f}")
print(f"Test Accuracy: {test_acc7:.3f}")

In [ ]:
model_name7 = "No StemLayer"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name7]


results.append({
    "Model": model_name7,
    "Test Loss": test_loss7,
    "Test Accuracy": test_acc7,
    "Test Precision": test_prec7,
    "Test Recall": test_recall7,
    "Test F1": test_f17
})

In [ ]:
all_histories[model_name7] = history7

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history7)
plot_loss(history7)
plot_learning_rate(history7)

## **Model variant: No Residual Blocks**

**Change**
- Replaced all residual blocks with plain convolutional blocks, removing skip connections while preserving depth and channel dimensions.

**Purpose**
- To evaluate the contribution of residual connections to optimization stability and final accuracy.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_8 = Net(n_classes=train_dataset.get_num_classes(), use_residuals=False).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_8.parameters())
trainable_params = sum(p.numel() for p in Model_8.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config8 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noRES_net.pth",
)

criterion8, optimizer8, scheduler8, mixup8 = build_training_components(
    model = Model_8,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config8,
)

best_acc8, history8 = train_model(
    model=Model_8,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion8,
    optimizer=optimizer8,
    scheduler=scheduler8,
    mixup=mixup8,
    device=device,
    num_epochs=config8.num_epochs,
    save_path=config8.save_path,
)

In [ ]:
best_model8 = Model_8

best_model8.load_state_dict(
    torch.load(config8.save_path, map_location=device)
)

best_model8.to(device)

test_loss8, test_acc8, test_prec8, test_recall8, test_f18 = evaluate(best_model8, test_loader, criterion8, device)

print(f"Test Loss: {test_loss8:.3f}")
print(f"Test Accuracy: {test_acc8:.3f}")

In [ ]:
model_name8 = "No Residual"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name8]


results.append({
    "Model": model_name8,
    "Test Loss": test_loss8,
    "Test Accuracy": test_acc8,
    "Test Precision": test_prec8,
    "Test Recall": test_recall8,
    "Test F1": test_f18
})

In [ ]:
all_histories[model_name8] = history8

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history8)
plot_loss(history8)
plot_learning_rate(history8)

## **Model variant: No Label Smoothing**

**Change**
- Disabled label smoothing in the cross-entropy loss, using hard one-hot target labels during training.

**Purpose**
- To assess the effect of label smoothing on model calibration and generalization.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_9 = Net(n_classes=train_dataset.get_num_classes()).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_9.parameters())
trainable_params = sum(p.numel() for p in Model_9.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config9 = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=False,
    use_mixup=True,
    save_path="best_noSMOOTH_net.pth",
)

criterion9, optimizer9, scheduler9, mixup9 = build_training_components(
    model = Model_9,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config9,
)

best_acc9, history9 = train_model(
    model=Model_9,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion9,
    optimizer=optimizer9,
    scheduler=scheduler9,
    mixup=mixup9,
    device=device,
    num_epochs=config9.num_epochs,
    save_path=config9.save_path,
)

In [ ]:
best_model9 = Model_9

best_model9.load_state_dict(
    torch.load(config9.save_path, map_location=device)
)

best_model9.to(device)

test_loss9, test_acc9, test_prec9, test_recall9, test_f19 = evaluate(best_model9, test_loader, criterion9, device)

print(f"Test Loss: {test_loss9:.3f}")
print(f"Test Accuracy: {test_acc9:.3f}")

In [ ]:
model_name9 = "No LabelSmoothing"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name9]


results.append({
    "Model": model_name9,
    "Test Loss": test_loss9,
    "Test Accuracy": test_acc9,
    "Test Precision": test_prec9,
    "Test Recall": test_recall9,
    "Test F1": test_f19
})

In [ ]:
all_histories[model_name9] = history9

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history9)
plot_loss(history9)
plot_learning_rate(history9)

## **Model variant: No Learning Rate Scheduler**

**Change**
- Disabled the learning rate scheduler, training the model with a constant learning rate throughout all epochs.

**Purpose**
- To evaluate the impact of dynamic learning rate scheduling on convergence speed and final performance.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Re-initialize model with fresh weights
Model_10 = Net(n_classes=train_dataset.get_num_classes()).to(device)

# Count parameters
total_params = sum(p.numel() for p in Model_10.parameters())
trainable_params = sum(p.numel() for p in Model_10.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
config10 = TrainConfig(
    use_scheduler=False,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_noLRSCHED_net.pth",
)

criterion10, optimizer10, scheduler10, mixup10 = build_training_components(
    model = Model_10,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config10,
)

best_acc10, history10 = train_model(
    model=Model_10,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion10,
    optimizer=optimizer10,
    scheduler=scheduler10,
    mixup=mixup10,
    device=device,
    num_epochs=config10.num_epochs,
    save_path=config10.save_path,
)

In [ ]:
best_model10 = Model_10

best_model10.load_state_dict(
    torch.load(config10.save_path, map_location=device)
)

best_model10.to(device)

test_loss10, test_acc10, test_prec10, test_recall10, test_f110 = evaluate(best_model10, test_loader, criterion10, device)

print(f"Test Loss: {test_loss10:.3f}")
print(f"Test Accuracy: {test_acc10:.3f}")

In [ ]:
model_name10 = "No LR Scheduler"

# Remove existing entry if re-training same model
results = [r for r in results if r["Model"] != model_name10]


results.append({
    "Model": model_name10,
    "Test Loss": test_loss10,
    "Test Accuracy": test_acc10,
    "Test Precision": test_prec10,
    "Test Recall": test_recall10,
    "Test F1": test_f110
})

In [ ]:
all_histories[model_name10] = history10

In [ ]:
# Save CSV
df_results = pd.DataFrame(results).round(4)
df_results.to_csv(CSV_PATH, index=False)

# Save JSON
with open(JSON_PATH, "w") as f:
    json.dump(all_histories, f, indent=4)

In [ ]:
plot_accuracy(history10)
plot_loss(history10)
plot_learning_rate(history10)

## **RECAP**



In [ ]:
model_order = [
    "Baseline",
    "No BatchNorm",
    "No Pooling",
    "No Dropout",
    "No MixUp",
    "No Augmentation",
    "No StemLayer",
    "No Residual",
    "No LabelSmoothing",
    "No LR Scheduler"
]

In [ ]:
plt.figure()

for model in model_order:
    if model in all_histories:
        plt.plot(all_histories[model]["val_acc"])

plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy - Ablation Study")
plt.legend([m for m in model_order if m in all_histories], loc="center left", bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()


In [ ]:
plt.figure()

for model in model_order:
    if model in all_histories:
        plt.plot(all_histories[model]["train_loss"])

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss - Ablation Study")
plt.legend([m for m in model_order if m in all_histories], loc="center left", bbox_to_anchor=(1, 0.5))
plt.grid(True)
plt.show()

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained ResNet-18 model on `OxfordPetDataset`. Use the implementation provided by PyTorch, i.e. the opposite of part 1. Specifically, use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

2A. First, fine-tune the ResNet-18 with the same training hyperparameters you used for your best model in part 1.

2B. Then, tweak the training hyperparameters in order to increase the accuracy on the test split. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions — papers, blog posts, YouTube videos, or whatever else you may find useful. You should consider yourselves satisfied once you obtain a classification accuracy on the test split of ~90%.

In [ ]:
# Part 2A: Fine-tune ResNet-18 with the same hyperparameters as Part 1

from torchvision import models
from torchvision.models import ResNet18_Weights

# Load pretrained ResNet-18 (ImageNet-1K V1 weights)
resnet18 = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Replace the final classifier layer for 37 pet classes
num_classes = train_dataset.get_num_classes()
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)

# Count parameters
total_params = sum(p.numel() for p in resnet18.parameters())
trainable_params = sum(p.numel() for p in resnet18.parameters() if p.requires_grad)
print(f"Using device: {device}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# Train ResNet-18 with the same config as Part 1 best model (full baseline)

config_2a = TrainConfig(
    use_scheduler=True,
    use_label_smoothing=True,
    use_mixup=True,
    save_path="best_resnet18_2a.pth",
)

criterion_2a, optimizer_2a, scheduler_2a, mixup_2a = build_training_components(
    model=resnet18,
    train_loader=train_loader,
    train_dataset=train_dataset,
    config=config_2a,
)

best_acc_2a, history_2a = train_model(
    model=resnet18,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion_2a,
    optimizer=optimizer_2a,
    scheduler=scheduler_2a,
    mixup=mixup_2a,
    device=device,
    num_epochs=config_2a.num_epochs,
    save_path=config_2a.save_path,
)

In [ ]:
# Evaluate ResNet-18 Part 2A on test set

resnet18.load_state_dict(
    torch.load(config_2a.save_path, map_location=device)
)
resnet18.to(device)

test_loss_2a, test_acc_2a, test_prec_2a, test_recall_2a, test_f1_2a = evaluate(resnet18, test_loader, criterion_2a, device)

print(f"Part 2A - ResNet-18 Fine-tuned (same hyperparameters as Part 1)")
print(f"Test Loss: {test_loss_2a:.3f}")
print(f"Test Accuracy: {test_acc_2a:.3f}")

In [ ]:
plot_accuracy(history_2a)
plot_loss(history_2a)
plot_learning_rate(history_2a)

## Part 2B: Fine-tune ResNet-18 with optimized hyperparameters

**Changes from Part 2A:**
- **Lower learning rate**: Pretrained models benefit from smaller learning rates (1e-4 to 3e-4) to avoid destroying pretrained features
- **Layer-wise learning rate decay**: Lower learning rate for earlier layers, higher for classifier (discriminative fine-tuning)
- **Longer warmup**: Gradual warmup helps stabilize training when fine-tuning
- **Freeze backbone initially** (optional): Train only the classifier first, then unfreeze the backbone

**Justification:**
- Transfer learning best practices suggest using smaller learning rates for pretrained layers (Yosinski et al., "How transferable are features in deep neural networks?", NeurIPS 2014)
- The pretrained features are already well-optimized for image classification; aggressive updates can degrade them

In [ ]:
# Part 2B: Fine-tune ResNet-18 with optimized hyperparameters

# Re-initialize pretrained ResNet-18
resnet18_2b = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
num_classes = train_dataset.get_num_classes()
resnet18_2b.fc = nn.Sequential(
    nn.Dropout(p=0.4, inplace=False),
    nn.Linear(in_features=512, out_features=num_classes, bias=True)
)

resnet18_2b = resnet18_2b.to(device)

# Training only the head
resnet18_2b.requires_grad_ = False
resnet18_2b.fc.requires_grad_ = True

print(f"Using device: {device}")
total_params = sum(p.numel() for p in resnet18_2b.parameters())
trainable_params = sum(p.numel() for p in resnet18_2b.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# Train with optimized hyperparameters for transfer learning
base_lr = 1e-3
classifier_lr = 1e-5
num_epochs_2b = 100

optimizer_fc = torch.optim.AdamW(resnet18_2b.fc.parameters(), lr=base_lr, weight_decay=1e-3)
optimizer_full = torch.optim.AdamW(resnet18_2b.parameters(), lr=classifier_lr, weight_decay=1e-3)

scheduler_fc = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_fc, T_max=num_epochs_2b)
scheduler_full = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_full, T_max=num_epochs_2b)

# Use label smoothing for regularization
criterion_2b = nn.CrossEntropyLoss(label_smoothing=0.05)

In [ ]:
# Train the model - only the head
best_acc_2b, history_2b = train_model(
    model=resnet18_2b,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion_2b,
    optimizer=optimizer_fc,
    scheduler=scheduler_fc,
    mixup=NoMixUp(),
    device=device,
    num_epochs=num_epochs_2b,
    save_path="best_resnet18_2b.pth",
)

In [ ]:
# Evaluate ResNet-18 Part 2B on test set

resnet18_2b.load_state_dict(
    torch.load("best_resnet18_2b.pth", map_location=device)
)
resnet18_2b.to(device)

test_loss_2b, test_acc_2b, test_prec_2b, test_recall_2b, test_f1_2b = evaluate(resnet18_2b, test_loader, criterion_2b, device)

print(f"Part 2B - ResNet-18 Fine-tuned - only the head (optimized hyperparameters)")
print(f"Test Loss: {test_loss_2b:.3f}")
print(f"Test Accuracy: {test_acc_2b:.3f}")

In [ ]:
plot_accuracy(history_2b)
plot_loss(history_2b)
plot_learning_rate(history_2b)

In [ ]:
# Training all
resnet18_2b.requires_grad_ = True
best_acc_2b2, history_2b2 = train_model(
    model=resnet18_2b,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion_2b,
    optimizer=optimizer_full,
    scheduler=scheduler_full,
    mixup=NoMixUp(),
    device=device,
    num_epochs=num_epochs_2b,
    save_path="best_resnet18_2b_full.pth",
)

In [ ]:
resnet18_2b.load_state_dict(
    torch.load("best_resnet18_2b_full.pth", map_location=device)
)
resnet18_2b.to(device)

test_loss_2b2, test_acc_2b2, test_prec_2b2, test_recall_2b2, test_f1_2b2 = evaluate(resnet18_2b, test_loader, criterion_2b, device)

print(f"Part 2B - ResNet-18 Fine-tuned (optimized hyperparameters)")
print(f"Test Loss: {test_loss_2b2:.3f}")
print(f"Test Accuracy: {test_acc_2b2:.3f}")

In [ ]:
# Summary comparison: Part 2A vs Part 2B
print("Part 2 Summary: ResNet-18 Fine-tuning Results")
print(f"\nPart 2A (same hyperparameters as Part 1):")
print(f"  Test Accuracy: {test_acc_2a:.3f}")
print(f"\nPart 2B (optimized hyperparameters for transfer learning):")
print(f"  Test Accuracy: {test_acc_2b2:.3f}")
print(f"\nImprovement: {(test_acc_2b2 - test_acc_2a) * 100:.2f} percentage points")